# Popular Movies Datasets

***🍿 About Dataset***

[Dataset](https://www.kaggle.com/datasets/whenamancodes/popular-movies-datasets-58000-movies) from Kaggle with a list of 58.098 Movies with 27.753.444 Ratings & 1.108.997 Tag Applications on 58.098 Movies by 280.000 Users. This dataset contains user information between January 09, 1995 - September 26, 2018.

***❔Questions***
1. What are the most popular genres each year?
2. Visualize trends between ratings, budgets, genres, and years.


***🧹 Performed data wrangling and cleaning.***

1. Seperate years in the movies dataset into their own column
2. Seperate genres into their own columns by One-Hot Encoding column
3. Remove timestamp column from ratings dataset since we will not be using it
4. Reduce the number of reviews to less than 15M to be able to utilize in Tableau

In [1]:
# Import libraries
from google.colab import drive
import pandas as pd

# Open drive to access dataset files
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
movies=pd.read_csv('/content/drive/MyDrive/Ana Garcia Work Folder/Project - Movie Tableau /archive/movies.csv')
ratings=pd.read_csv('/content/drive/MyDrive/Ana Garcia Work Folder/Project - Movie Tableau /archive/ratings.csv')


# Look into the two datasets we will be using

[Overview Link](https://files.grouplens.org/datasets/movielens/ml-latest-README.html)

**Movies Data File Structure (movies.csv)**<br>
Movie information is contained in the file movies.csv. Each line of this file after the header row represents one movie, and has the following format:

*movieId,title,genres*

Movie titles are entered manually or imported from https://www.themoviedb.org/, and include the year of release in parentheses. Errors and inconsistencies may exist in these titles.

Genres are a pipe-separated list, and are selected from the following:

* Action
* Adventure
* Animation
* Children's
* Comedy
* Crime
* Documentary
* Drama
* Fantasy
* Film-Noir
* Horror
* Musical
* Mystery
* Romance
* Sci-Fi
* Thriller
* War
* Western
* (no genres listed)
<br>
<br>

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


**Ratings Data File Structure (ratings.csv)**<br>
All ratings are contained in the file ratings.csv. Each line of this file after the header row represents one rating of one movie by one user, and has the following format:

*userId,movieId,rating,timestamp*

The lines within this file are ordered first by userId, then, within user, by movieId.

Ratings are made on a 5-star scale, with half-star increments (0.5 stars - 5.0 stars).

Timestamps represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970.



In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [6]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


# Clean the data

***Movies Dataset***

In [7]:
import re

def extract_year(text):
    # Find any 4-digit number inside parentheses
    matches = re.findall(r'\((\d{4})\)', text)
    if matches:
        return int(matches[-1])  # usually the last one is the year
    return None


In [8]:
# Extract release year into a new column using the extract_year function
# Do not convert to float
movies["release_year"] = movies["title"].apply(extract_year).astype("Int64")

In [9]:
movies.head()

,movieId,title,genres,release_year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [10]:
# Look at row 29
movies.iloc[29]

,29
movieId,30
title,Shanghai Triad (Yao a yao yao dao waipo qiao) ...
genres,Crime|Drama
release_year,1995


In [11]:
# Seperate genres into their own columns with 0s and 1s (One-Hot Encoding)
# movies_encoded = movies['genres'].str.split('|').str.get_dummies()
movies['genres_split'] = movies['genres'].str.split('|')

In [12]:
movies.head()

,movieId,title,genres,release_year,genres_split
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),Comedy,1995,[Comedy]


In [13]:
# Get the genres and one-hot encode them
# Create a loop that one-hot encodes genres based if the specific genre is in the vector
genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', '(no genres listed)']

for genre in genres:
    movies[genre] = movies['genres_split'].apply(lambda x: 1 if genre in x else 0)

In [14]:
movies.head()

,movieId,title,genres,release_year,genres_split,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,(no genres listed)
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995,"[Adventure, Animation, Children, Comedy, Fantasy]",0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995,"[Adventure, Children, Fantasy]",0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,1995,"[Comedy, Romance]",0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995,"[Comedy, Drama, Romance]",0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,1995,[Comedy],0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Show how many rows and col
movies.shape

(58098, 24)

In [16]:
# Show how many movies have 'genre not listed'
movies['(no genres listed)'].sum()

np.int64(4266)

In [17]:
# Remove movies if genre is not listed
movies = movies[movies['(no genres listed)'] == 0]

In [18]:
movies.shape

(53832, 24)

***Ratings Dataset***

In [19]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [20]:
# Drop the timestamp column
ratings.drop('timestamp', axis=1, inplace=True)

In [21]:
ratings.columns

Index(['userId', 'movieId', 'rating'], dtype='object')

In [22]:
ratings.shape

(27753444, 3)

In [23]:
# Remove ratings where movieId does not exist
ratings = ratings[ratings['movieId'].isin(movies['movieId'])]

In [24]:
ratings.shape

(27735055, 3)

In [25]:
# Each user can only have approx 300 reviews, if they exceed remove only the exceeded amount
ratings_counts = ratings['userId'].value_counts()

print(ratings_counts)
ratings_counts.head()

userId
123100    22639
117490     9277
134596     8303
212343     7862
242683     7515
          ...  
13716         1
81463         1
7309          1
7308          1
33            1
Name: count, Length: 283220, dtype: int64


,count
userId,
123100,22639
117490,9277
134596,8303
212343,7862
242683,7515


In [26]:
ratings_counts = ratings['userId'].value_counts()

# Identify users who have more than 100 ratings
users_to_limit = ratings_counts[ratings_counts > 100].index

# Create a list to store the processed dataframes
limited_ratings_list = []

# Iterate through each user and limit their ratings to 100 if they exceed
for user_id in users_to_limit:
    user_ratings = ratings[ratings['userId'] == user_id]
    # Keep only the first 100 ratings for this user
    limited_ratings_list.append(user_ratings.head(100))

# Get ratings from users who do not exceed the limit
users_not_limited = ratings_counts[ratings_counts <= 100].index
ratings_not_limited = ratings[ratings['userId'].isin(users_not_limited)]

# Concatenate all ratings back into a single DataFrame
ratings = pd.concat(limited_ratings_list + [ratings_not_limited])

print(f"New ratings shape: {ratings.shape}")
print(f"Maximum ratings per user: {ratings['userId'].value_counts().max()}")

New ratings shape: (12990957, 3)
Maximum ratings per user: 100


In [27]:
ratings.head()

,userId,movieId,rating
11998477,123100,1,4.0
11998478,123100,2,3.0
11998479,123100,5,2.5
11998480,123100,6,4.0
11998481,123100,7,3.0


In [28]:
# If a movieId has no ratings then remove it from the movie list
movies = movies[movies['movieId'].isin(ratings['movieId'])]

In [29]:
movies.shape

(23725, 24)

In [30]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23725 entries, 0 to 58097
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   movieId             23725 non-null  int64 
 1   title               23725 non-null  object
 2   genres              23725 non-null  object
 3   release_year        23696 non-null  Int64 
 4   genres_split        23725 non-null  object
 5   Action              23725 non-null  int64 
 6   Adventure           23725 non-null  int64 
 7   Animation           23725 non-null  int64 
 8   Children            23725 non-null  int64 
 9   Comedy              23725 non-null  int64 
 10  Crime               23725 non-null  int64 
 11  Documentary         23725 non-null  int64 
 12  Drama               23725 non-null  int64 
 13  Fantasy             23725 non-null  int64 
 14  Film-Noir           23725 non-null  int64 
 15  Horror              23725 non-null  int64 
 16  Musical             23725 n

In [31]:
# Drop the no genres listed table
movies.drop('(no genres listed)', axis=1, inplace=True)

In [32]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23725 entries, 0 to 58097
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieId       23725 non-null  int64 
 1   title         23725 non-null  object
 2   genres        23725 non-null  object
 3   release_year  23696 non-null  Int64 
 4   genres_split  23725 non-null  object
 5   Action        23725 non-null  int64 
 6   Adventure     23725 non-null  int64 
 7   Animation     23725 non-null  int64 
 8   Children      23725 non-null  int64 
 9   Comedy        23725 non-null  int64 
 10  Crime         23725 non-null  int64 
 11  Documentary   23725 non-null  int64 
 12  Drama         23725 non-null  int64 
 13  Fantasy       23725 non-null  int64 
 14  Film-Noir     23725 non-null  int64 
 15  Horror        23725 non-null  int64 
 16  Musical       23725 non-null  int64 
 17  Mystery       23725 non-null  int64 
 18  Romance       23725 non-null  int64 
 19  Sci-Fi   

In [33]:
ratings.head()

,userId,movieId,rating
11998477,123100,1,4.0
11998478,123100,2,3.0
11998479,123100,5,2.5
11998480,123100,6,4.0
11998481,123100,7,3.0


In [36]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23725 entries, 0 to 58097
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   movieId       23725 non-null  int64 
 1   title         23725 non-null  object
 2   genres        23725 non-null  object
 3   release_year  23696 non-null  Int64 
 4   genres_split  23725 non-null  object
 5   Action        23725 non-null  int64 
 6   Adventure     23725 non-null  int64 
 7   Animation     23725 non-null  int64 
 8   Children      23725 non-null  int64 
 9   Comedy        23725 non-null  int64 
 10  Crime         23725 non-null  int64 
 11  Documentary   23725 non-null  int64 
 12  Drama         23725 non-null  int64 
 13  Fantasy       23725 non-null  int64 
 14  Film-Noir     23725 non-null  int64 
 15  Horror        23725 non-null  int64 
 16  Musical       23725 non-null  int64 
 17  Mystery       23725 non-null  int64 
 18  Romance       23725 non-null  int64 
 19  Sci-Fi   

# Download datasets

In [34]:
movies.to_csv('movies_new.csv', index=False) # Save the DataFrame to a CSV file
ratings.to_csv('ratings_new.csv', index=False) # Save the DataFrame to a CSV file

In [35]:
from google.colab import files

files.download('movies_new.csv') # Initiate the download
files.download('ratings_new.csv') # Initiate the download

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>